  ### BaseBall Most Valuable Player
<img src="notebooks/bb.jpg">

In [1]:

## Import dependencies here

import os
import datetime
import numpy as np
import pandas as pd
from matplotlib import pyplot
from pprint import pprint

# current date/time
now = datetime.datetime.now()

### Data Acquisition

In [2]:
# Read Baseball datasets into individual dataframes

# location of baseball datasets
dataset_path = "data/mvp_baseball_data/core/"

# load Player data into the dataframe
master_df = pd.read_csv(os.path.join(dataset_path, 'Master.csv'),
                        usecols=['playerID','nameFirst','nameLast','bats','throws','debut','finalGame', 'deathYear'])

# load Fielding data into the dataframe
fielding_df = pd.read_csv(os.path.join(dataset_path, 'Fielding.csv'),
                          usecols=['playerID','yearID','stint','teamID','lgID','POS','G','GS','InnOuts','PO','A','E','DP'])

# load Batting data into the dataframe
batting_df = pd.read_csv(os.path.join(dataset_path, 'Batting.csv'))

# load Pitching data into the dataframe
pitching_df = pd.read_csv(os.path.join(dataset_path, 'Pitching.csv'))

# load Awards data into the dataframe
awards_df = pd.read_csv(os.path.join(dataset_path, 'AwardsPlayers.csv'), 
                        usecols=['playerID','awardID','yearID'])

# load All-Star category data into the dataframe
allstar_df = pd.read_csv(os.path.join(dataset_path, 'AllstarFull.csv'),
                         usecols=['playerID','yearID'])

# load Appearances in Baseball match into the dataframe
appearances_df = pd.read_csv(os.path.join(dataset_path, 'Appearances.csv'))


### Data Cleanup & Preparation

In [3]:
# Eliminate players those are not alive any more
master_df = master_df[master_df['deathYear'].isnull()]
master_df = master_df.drop(columns=['deathYear'])
master_df.head()

,playerID,nameFirst,nameLast,bats,throws,debut,finalGame
0,aardsda01,David,Aardsma,R,R,2004-04-06,2015-08-23
1,aaronha01,Hank,Aaron,R,R,1954-04-13,1976-10-03
3,aasedo01,Don,Aase,R,R,1977-07-26,1990-10-03
4,abadan01,Andy,Abad,L,L,2001-09-10,2006-04-13
5,abadfe01,Fernando,Abad,L,L,2010-07-28,2016-09-25


In [4]:
# batting stats
batting_df.head()

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,abercda01,1871,1,TRO,NaN,1,4,0,0,0,...,0.0,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN
1,addybo01,1871,1,RC1,NaN,25,118,30,32,6,...,13.0,8.0,1.0,4,0.0,NaN,NaN,NaN,NaN,NaN
2,allisar01,1871,1,CL1,NaN,29,137,28,40,4,...,19.0,3.0,1.0,2,5.0,NaN,NaN,NaN,NaN,NaN
3,allisdo01,1871,1,WS3,NaN,27,133,28,44,10,...,27.0,1.0,1.0,0,2.0,NaN,NaN,NaN,NaN,NaN
4,ansonca01,1871,1,RC1,NaN,25,120,29,39,11,...,16.0,6.0,2.0,2,1.0,NaN,NaN,NaN,NaN,NaN


In [5]:
# Initialize dictionaries for player stats and years played
player_stats = {}
years_played = {}

# Create dictionaries for player stats and years played from `batting_df`
for i, row in batting_df.iterrows():
    playerID = row['playerID']
    if playerID in player_stats:
        player_stats[playerID]['G'] = player_stats[playerID]['G'] + row['G']
        player_stats[playerID]['AB'] = player_stats[playerID]['AB'] + row['AB']
        player_stats[playerID]['R'] = player_stats[playerID]['R'] + row['R']
        player_stats[playerID]['H'] = player_stats[playerID]['H'] + row['H']
        player_stats[playerID]['2B'] = player_stats[playerID]['2B'] + row['2B']
        player_stats[playerID]['3B'] = player_stats[playerID]['3B'] + row['3B']
        player_stats[playerID]['HR'] = player_stats[playerID]['HR'] + row['HR']
        player_stats[playerID]['RBI'] = player_stats[playerID]['RBI'] + row['RBI']
        player_stats[playerID]['SB'] = player_stats[playerID]['SB'] + row['SB']
        player_stats[playerID]['BB'] = player_stats[playerID]['BB'] + row['BB']
        player_stats[playerID]['SO'] = player_stats[playerID]['SO'] + row['SO']
        player_stats[playerID]['IBB'] = player_stats[playerID]['IBB'] + row['IBB']
        player_stats[playerID]['HBP'] = player_stats[playerID]['HBP'] + row['HBP']
        player_stats[playerID]['SH'] = player_stats[playerID]['SH'] + row['SH']
        player_stats[playerID]['SF'] = player_stats[playerID]['SF'] + row['SF']
        years_played[playerID].append(row['yearID'])
    else:
        player_stats[playerID] = {}
        player_stats[playerID]['G'] = row['G']
        player_stats[playerID]['AB'] = row['AB']
        player_stats[playerID]['R'] = row['R']
        player_stats[playerID]['H'] = row['H']
        player_stats[playerID]['2B'] = row['2B']
        player_stats[playerID]['3B'] = row['3B']
        player_stats[playerID]['HR'] = row['HR']
        player_stats[playerID]['RBI'] = row['RBI']
        player_stats[playerID]['SB'] = row['SB']
        player_stats[playerID]['BB'] = row['BB']
        player_stats[playerID]['SO'] = row['SO']
        player_stats[playerID]['IBB'] = row['IBB']
        player_stats[playerID]['HBP'] = row['HBP']
        player_stats[playerID]['SH'] = row['SH']
        player_stats[playerID]['SF'] = row['SF']
        years_played[playerID] = []
        years_played[playerID].append(row['yearID'])

In [6]:
# Iterate through `years_played` and add the number of years played to `player_stats`
for k, v in years_played.items():
    player_stats[k]['Years_Played'] = len(list(set(v)))

In [7]:
# Initialize `fielder_list`
fielder_list = []

# Add fielding stats to `player_stats` from `fielding_df`
for i, row in fielding_df.iterrows():
    playerID = row['playerID']
    Gf = row['G']
    GSf = row['GS']
    POf = row['PO']
    Af = row['A']
    Ef = row['E']
    DPf = row['DP']
    if playerID in player_stats and playerID in fielder_list:
        player_stats[playerID]['Gf'] = player_stats[playerID]['Gf'] + Gf
        player_stats[playerID]['GSf'] = player_stats[playerID]['GSf'] + GSf
        player_stats[playerID]['POf'] = player_stats[playerID]['POf'] + POf
        player_stats[playerID]['Af'] = player_stats[playerID]['Af'] + Af
        player_stats[playerID]['Ef'] = player_stats[playerID]['Ef'] + Ef
        player_stats[playerID]['DPf'] = player_stats[playerID]['DPf'] + DPf
    else:
        fielder_list.append(playerID)
        player_stats[playerID]['Gf'] = Gf
        player_stats[playerID]['GSf'] = GSf
        player_stats[playerID]['POf'] = POf
        player_stats[playerID]['Af'] = Af
        player_stats[playerID]['Ef'] = Ef
        player_stats[playerID]['DPf'] = DPf


In [8]:
# Create DataFrames for players who already won Most Valuable Player Award
awards_list = [awards_df[awards_df['awardID'] == 'Most Valuable Player']]

# Initialize lists for each of the above DataFrames
mvp_list = []

# Include each of the above lists in `lists`
lists = [mvp_list]

# Add a count for each award for each player in `player_stats`
for index, v in enumerate(awards_list):
    for i, row in v.iterrows():
        playerID = row['playerID']
        award = row['awardID']
        if playerID in player_stats and playerID in lists[index]:
            player_stats[playerID][award] += 1
        else:
            lists[index].append(playerID)
            player_stats[playerID][award] = 1

In [9]:
# Initialize `allstar_list`
allstar_list = []

# Add a count for each Allstar game appearance for each player in `player_stats`
for i, row in allstar_df.iterrows():
    playerID = row['playerID']
    if playerID in player_stats and playerID in allstar_list:
        player_stats[playerID]['AS_games'] += 1
    else:
        allstar_list.append(playerID)
        player_stats[playerID]['AS_games'] = 1

In [10]:
# Convert `player_stats` into a DataFrame
stats_df = pd.DataFrame.from_dict(player_stats, orient='index')

# player stats
stats_df

,G,AB,R,H,2B,3B,HR,RBI,SB,BB,...,SF,Years_Played,Gf,GSf,POf,Af,Ef,DPf,Most Valuable Player,AS_games
aardsda01,331,4,0,0,0,0,0,0.0,0.0,0,...,0.0,9,331.0,0.0,11.0,29.0,3.0,2.0,NaN,NaN
aaronha01,3298,12364,2174,3771,624,98,755,2297.0,240.0,1402,...,121.0,23,3020.0,2977.0,7436.0,429.0,144.0,218.0,1.0,25.0
aaronto01,437,944,102,216,42,6,13,94.0,9.0,86,...,6.0,7,387.0,206.0,1317.0,113.0,22.0,124.0,NaN,NaN
aasedo01,448,5,0,0,0,0,0,0.0,0.0,0,...,0.0,13,448.0,91.0,67.0,135.0,13.0,10.0,NaN,1.0
abadan01,15,21,1,2,0,0,0,0.0,0.0,4,...,0.0,3,9.0,4.0,37.0,1.0,1.0,3.0,NaN,NaN
abadfe01,315,9,0,1,0,0,0,0.0,0.0,0,...,0.0,7,315.0,6.0,7.0,31.0,2.0,1.0,NaN,NaN
abadijo01,12,49,4,11,0,0,0,5.0,1.0,0,...,NaN,1,12.0,NaN,129.0,3.0,13.0,1.0,NaN,NaN
abbated01,855,3044,355,772,99,43,11,324.0,142.0,289,...,NaN,9,830.0,NaN,1872.0,2368.0,315.0,247.0,NaN,NaN
abbeybe01,79,225,21,38,3,3,0,17.0,3.0,21,...,NaN,5,79.0,NaN,17.0,134.0,22.0,4.0,NaN,NaN
abbeych01,451,1751,307,492,67,46,19,280.0,93.0,167,...,NaN,5,451.0,NaN,917.0,90.0,99.0,18.0,NaN,NaN


In [11]:
# Add a column for playerID from the `stats_df` index
stats_df['playerID'] = stats_df.index

# Join `stats_df` and `master_df`
master_df = master_df.join(stats_df,on='playerID',how='inner', rsuffix='mstr')

# Inspect first rows of `master_df`
master_df.head()

,playerID,nameFirst,nameLast,bats,throws,debut,finalGame,G,AB,R,...,Years_Played,Gf,GSf,POf,Af,Ef,DPf,Most Valuable Player,AS_games,playerIDmstr
0,aardsda01,David,Aardsma,R,R,2004-04-06,2015-08-23,331,4,0,...,9,331.0,0.0,11.0,29.0,3.0,2.0,NaN,NaN,aardsda01
1,aaronha01,Hank,Aaron,R,R,1954-04-13,1976-10-03,3298,12364,2174,...,23,3020.0,2977.0,7436.0,429.0,144.0,218.0,1.0,25.0,aaronha01
3,aasedo01,Don,Aase,R,R,1977-07-26,1990-10-03,448,5,0,...,13,448.0,91.0,67.0,135.0,13.0,10.0,NaN,1.0,aasedo01
4,abadan01,Andy,Abad,L,L,2001-09-10,2006-04-13,15,21,1,...,3,9.0,4.0,37.0,1.0,1.0,3.0,NaN,NaN,abadan01
5,abadfe01,Fernando,Abad,L,L,2010-07-28,2016-09-25,315,9,0,...,7,315.0,6.0,7.0,31.0,2.0,1.0,NaN,NaN,abadfe01


In [12]:
# Initialize a dictionary
pos_dict = {}

# Iterate through `appearances_df`
# Add a count for the number of appearances for each player at each position
# Also add a count for the number of games played for each player in each era.
for i, row in appearances_df.iterrows():
    ID = row['playerID']
    year = row['yearID']
    if ID in pos_dict:
        pos_dict[ID]['G_all'] = pos_dict[ID]['G_all'] + row['G_all']
        pos_dict[ID]['G_p'] = pos_dict[ID]['G_p'] + row['G_p']
        pos_dict[ID]['G_c'] = pos_dict[ID]['G_c'] + row['G_c']
        pos_dict[ID]['G_1b'] = pos_dict[ID]['G_1b'] + row['G_1b']
        pos_dict[ID]['G_2b'] = pos_dict[ID]['G_2b'] + row['G_2b']
        pos_dict[ID]['G_3b'] = pos_dict[ID]['G_3b'] + row['G_3b']
        pos_dict[ID]['G_ss'] = pos_dict[ID]['G_ss'] + row['G_ss']
        pos_dict[ID]['G_lf'] = pos_dict[ID]['G_lf'] + row['G_lf']
        pos_dict[ID]['G_cf'] = pos_dict[ID]['G_cf'] + row['G_cf']
        pos_dict[ID]['G_rf'] = pos_dict[ID]['G_rf'] + row['G_rf']
        pos_dict[ID]['G_of'] = pos_dict[ID]['G_of'] + row['G_of']
        pos_dict[ID]['G_dh'] = pos_dict[ID]['G_dh'] + row['G_dh']
    else:
        pos_dict[ID] = {}
        pos_dict[ID]['G_all'] = row['G_all']
        pos_dict[ID]['G_p'] = row['G_p']
        pos_dict[ID]['G_c'] = row['G_c']
        pos_dict[ID]['G_1b'] = row['G_1b']
        pos_dict[ID]['G_2b'] = row['G_2b']
        pos_dict[ID]['G_3b'] = row['G_3b']
        pos_dict[ID]['G_ss'] = row['G_ss']
        pos_dict[ID]['G_lf'] = row['G_lf']
        pos_dict[ID]['G_cf'] = row['G_cf']
        pos_dict[ID]['G_rf'] = row['G_rf']
        pos_dict[ID]['G_of'] = row['G_of']
        pos_dict[ID]['G_dh'] = row['G_dh']

In [13]:
# Convert the `pos_dict` to a DataFrame
pos_df = pd.DataFrame.from_dict(pos_dict, orient='index')

# players played in various positions
pos_df.head()

,G_all,G_p,G_c,G_1b,G_2b,G_3b,G_ss,G_lf,G_cf,G_rf,G_of,G_dh
aardsda01,331,331,0,0,0,0,0,0,0,0,0,0.0
aaronha01,3298,0,0,210,43,7,0,315,308,2174,2760,201.0
aaronto01,437,0,0,232,7,10,0,135,1,2,137,0.0
aasedo01,448,448,0,0,0,0,0,0,0,0,0,0.0
abadan01,15,0,0,8,0,0,0,0,0,1,1,0.0


In [14]:
# Create a list from the columns of `pos_df`
pos_col_list = pos_df.columns.tolist()

# Remove the string 'G_all'
pos_col_list.remove('G_all')

# Loop through the list and divide each column by the players total games played
for col in pos_col_list:
    column = col + '_percent'
    pos_df[column] = pos_df[col] / pos_df['G_all']

# Print out the first rows of `pos_df`    
pos_df.head()

,G_all,G_p,G_c,G_1b,G_2b,G_3b,G_ss,G_lf,G_cf,G_rf,...,G_c_percent,G_1b_percent,G_2b_percent,G_3b_percent,G_ss_percent,G_lf_percent,G_cf_percent,G_rf_percent,G_of_percent,G_dh_percent
aardsda01,331,331,0,0,0,0,0,0,0,0,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
aaronha01,3298,0,0,210,43,7,0,315,308,2174,...,0.0,0.063675,0.013038,0.002122,0.0,0.095512,0.093390,0.659187,0.836871,0.060946
aaronto01,437,0,0,232,7,10,0,135,1,2,...,0.0,0.530892,0.016018,0.022883,0.0,0.308924,0.002288,0.004577,0.313501,0.000000
aasedo01,448,448,0,0,0,0,0,0,0,0,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
abadan01,15,0,0,8,0,0,0,0,0,1,...,0.0,0.533333,0.000000,0.000000,0.0,0.000000,0.000000,0.066667,0.066667,0.000000


In [15]:
# Filter `pos_df` to eliminate players who played 10% or more of their games as Pitchers or Catchers
pos_df = pos_df[(pos_df['G_p_percent'] < 0.1) & (pos_df['G_c_percent'] < 0.1)]

# Get info on `pos_df`
pos_df.head()

,G_all,G_p,G_c,G_1b,G_2b,G_3b,G_ss,G_lf,G_cf,G_rf,...,G_c_percent,G_1b_percent,G_2b_percent,G_3b_percent,G_ss_percent,G_lf_percent,G_cf_percent,G_rf_percent,G_of_percent,G_dh_percent
aaronha01,3298,0,0,210,43,7,0,315,308,2174,...,0.0,0.063675,0.013038,0.002122,0.000000,0.095512,0.093390,0.659187,0.836871,0.060946
aaronto01,437,0,0,232,7,10,0,135,1,2,...,0.0,0.530892,0.016018,0.022883,0.000000,0.308924,0.002288,0.004577,0.313501,0.000000
abadan01,15,0,0,8,0,0,0,0,0,1,...,0.0,0.533333,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.066667,0.000000
abadijo01,12,0,0,12,0,0,0,0,0,0,...,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
abbated01,855,0,0,0,419,20,388,0,2,1,...,0.0,0.000000,0.490058,0.023392,0.453801,0.000000,0.002339,0.001170,0.003509,NaN


In [16]:
# Join `pos_df` and `master_df`
master_df = master_df.join(pos_df,on='playerID',how='right')

# master_df at this stage after the cleanup and data preparation
master_df.head()

,playerID,nameFirst,nameLast,bats,throws,debut,finalGame,G,AB,R,...,G_c_percent,G_1b_percent,G_2b_percent,G_3b_percent,G_ss_percent,G_lf_percent,G_cf_percent,G_rf_percent,G_of_percent,G_dh_percent
1,aaronha01,Hank,Aaron,R,R,1954-04-13,1976-10-03,3298.0,12364.0,2174.0,...,0.0,0.063675,0.013038,0.002122,0.000000,0.095512,0.093390,0.659187,0.836871,0.060946
4,abadan01,Andy,Abad,L,L,2001-09-10,2006-04-13,15.0,21.0,1.0,...,0.0,0.533333,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.066667,0.000000
13,abbotje01,Jeff,Abbott,R,L,1997-06-10,2001-09-29,233.0,596.0,82.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.270386,0.347639,0.236052,0.793991,0.051502
15,abbotku01,Kurt,Abbott,R,R,1993-09-07,2001-04-13,702.0,2044.0,273.0,...,0.0,0.011396,0.246439,0.061254,0.497151,0.045584,0.005698,0.011396,0.061254,0.008547
21,abercre01,Reggie,Abercrombie,R,R,2006-04-04,2008-09-28,180.0,386.0,65.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.100000,0.638889,0.083333,0.744444,0.000000


In [17]:
# Create a function to convert the bats and throws colums to numeric
def bats_throws(col):
    if col == "R":
        return 1
    else:
        return 0

# Use the `apply()` method to create numeric columns from the bats and throws columns
master_df['bats_R'] = master_df['bats'].apply(bats_throws)
master_df['throws_R'] = master_df['throws'].apply(bats_throws)

# Print out the first rows of `master_df`
master_df.head()

,playerID,nameFirst,nameLast,bats,throws,debut,finalGame,G,AB,R,...,G_2b_percent,G_3b_percent,G_ss_percent,G_lf_percent,G_cf_percent,G_rf_percent,G_of_percent,G_dh_percent,bats_R,throws_R
1,aaronha01,Hank,Aaron,R,R,1954-04-13,1976-10-03,3298.0,12364.0,2174.0,...,0.013038,0.002122,0.000000,0.095512,0.093390,0.659187,0.836871,0.060946,1,1
4,abadan01,Andy,Abad,L,L,2001-09-10,2006-04-13,15.0,21.0,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.066667,0.000000,0,0
13,abbotje01,Jeff,Abbott,R,L,1997-06-10,2001-09-29,233.0,596.0,82.0,...,0.000000,0.000000,0.000000,0.270386,0.347639,0.236052,0.793991,0.051502,1,0
15,abbotku01,Kurt,Abbott,R,R,1993-09-07,2001-04-13,702.0,2044.0,273.0,...,0.246439,0.061254,0.497151,0.045584,0.005698,0.011396,0.061254,0.008547,1,1
21,abercre01,Reggie,Abercrombie,R,R,2006-04-04,2008-09-28,180.0,386.0,65.0,...,0.000000,0.000000,0.000000,0.100000,0.638889,0.083333,0.744444,0.000000,1,1


In [18]:
# Import datetime 
from datetime import datetime

# Convert the `debut` column to datetime
master_df['debut'] =  pd.to_datetime(master_df['debut'])
# Convert the `finalGame` column to datetime
master_df['finalGame'] = pd.to_datetime(master_df['finalGame'])

# Create new columns for debutYear and finalYear
master_df['debutYear'] = pd.to_numeric(master_df['debut'].dt.strftime('%Y'), errors='coerce')
master_df['finalYear'] = pd.to_numeric(master_df['finalGame'].dt.strftime('%Y'), errors='coerce')

# Return the first rows of `master_df`
master_df.head()

,playerID,nameFirst,nameLast,bats,throws,debut,finalGame,G,AB,R,...,G_ss_percent,G_lf_percent,G_cf_percent,G_rf_percent,G_of_percent,G_dh_percent,bats_R,throws_R,debutYear,finalYear
1,aaronha01,Hank,Aaron,R,R,1954-04-13,1976-10-03,3298.0,12364.0,2174.0,...,0.000000,0.095512,0.093390,0.659187,0.836871,0.060946,1,1,1954.0,1976.0
4,abadan01,Andy,Abad,L,L,2001-09-10,2006-04-13,15.0,21.0,1.0,...,0.000000,0.000000,0.000000,0.066667,0.066667,0.000000,0,0,2001.0,2006.0
13,abbotje01,Jeff,Abbott,R,L,1997-06-10,2001-09-29,233.0,596.0,82.0,...,0.000000,0.270386,0.347639,0.236052,0.793991,0.051502,1,0,1997.0,2001.0
15,abbotku01,Kurt,Abbott,R,R,1993-09-07,2001-04-13,702.0,2044.0,273.0,...,0.497151,0.045584,0.005698,0.011396,0.061254,0.008547,1,1,1993.0,2001.0
21,abercre01,Reggie,Abercrombie,R,R,2006-04-04,2008-09-28,180.0,386.0,65.0,...,0.000000,0.100000,0.638889,0.083333,0.744444,0.000000,1,1,2006.0,2008.0


In [19]:
# Eliminating unnecessary columns
df = master_df.drop(['IBB', 'bats', 'throws', 'GSf', 'POf','Gf', 'playerIDmstr'], axis=1)

# master_df with only the required columns
df.head()

,playerID,nameFirst,nameLast,debut,finalGame,G,AB,R,H,2B,...,G_ss_percent,G_lf_percent,G_cf_percent,G_rf_percent,G_of_percent,G_dh_percent,bats_R,throws_R,debutYear,finalYear
1,aaronha01,Hank,Aaron,1954-04-13,1976-10-03,3298.0,12364.0,2174.0,3771.0,624.0,...,0.000000,0.095512,0.093390,0.659187,0.836871,0.060946,1,1,1954.0,1976.0
4,abadan01,Andy,Abad,2001-09-10,2006-04-13,15.0,21.0,1.0,2.0,0.0,...,0.000000,0.000000,0.000000,0.066667,0.066667,0.000000,0,0,2001.0,2006.0
13,abbotje01,Jeff,Abbott,1997-06-10,2001-09-29,233.0,596.0,82.0,157.0,33.0,...,0.000000,0.270386,0.347639,0.236052,0.793991,0.051502,1,0,1997.0,2001.0
15,abbotku01,Kurt,Abbott,1993-09-07,2001-04-13,702.0,2044.0,273.0,523.0,109.0,...,0.497151,0.045584,0.005698,0.011396,0.061254,0.008547,1,1,1993.0,2001.0
21,abercre01,Reggie,Abercrombie,2006-04-04,2008-09-28,180.0,386.0,65.0,86.0,20.0,...,0.000000,0.100000,0.638889,0.083333,0.744444,0.000000,1,1,2006.0,2008.0


In [20]:
# Fill null values in numeric columns with 0
fill_cols = ['AS_games', 'Most Valuable Player', 'HBP', 'SB', 'SF', 'SH', 'RBI', 'SO', 'G_dh_percent', 'G_dh', 'Af', 'DPf', 'Ef']

for col in fill_cols:
    df[col] = df[col].fillna(0)

In [21]:
# Create Batting Average (`AVE`) column
df['AVE'] = df['H'] / df['AB']

# Create On Base Percent (`OBP`) column
plate_appearances = (df['AB'] + df['BB'] + df['SF'] + df['SH'] + df['HBP'])
df['OBP'] = (df['H'] + df['BB'] + df['HBP']) / plate_appearances

# Create Slugging Percent (`Slug_Percent`) column
single = ((df['H'] - df['2B']) - df['3B']) - df['HR']
df['Slug_Percent'] = ((df['HR'] * 4) + (df['3B'] * 3) + (df['2B'] * 2) + single) / df['AB']

# Create On Base plus Slugging Percent (`OPS`) column
hr = df['HR'] * 4
triple = df['3B'] * 3
double = df['2B'] * 2
df['OPS'] = df['OBP'] + df['Slug_Percent']

# Double check the `df` columns 
print(df.columns)

Index(['playerID', 'nameFirst', 'nameLast', 'debut', 'finalGame', 'G', 'AB',
       'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'BB', 'SO', 'HBP', 'SH', 'SF',
       'Years_Played', 'Af', 'Ef', 'DPf', 'Most Valuable Player', 'AS_games',
       'G_all', 'G_p', 'G_c', 'G_1b', 'G_2b', 'G_3b', 'G_ss', 'G_lf', 'G_cf',
       'G_rf', 'G_of', 'G_dh', 'G_p_percent', 'G_c_percent', 'G_1b_percent',
       'G_2b_percent', 'G_3b_percent', 'G_ss_percent', 'G_lf_percent',
       'G_cf_percent', 'G_rf_percent', 'G_of_percent', 'G_dh_percent',
       'bats_R', 'throws_R', 'debutYear', 'finalYear', 'AVE', 'OBP',
       'Slug_Percent', 'OPS'],
      dtype='object')


In [22]:
df.head()

,playerID,nameFirst,nameLast,debut,finalGame,G,AB,R,H,2B,...,G_of_percent,G_dh_percent,bats_R,throws_R,debutYear,finalYear,AVE,OBP,Slug_Percent,OPS
1,aaronha01,Hank,Aaron,1954-04-13,1976-10-03,3298.0,12364.0,2174.0,3771.0,624.0,...,0.836871,0.060946,1,1,1954.0,1976.0,0.304998,0.373386,0.554513,0.927899
4,abadan01,Andy,Abad,2001-09-10,2006-04-13,15.0,21.0,1.0,2.0,0.0,...,0.066667,0.000000,0,0,2001.0,2006.0,0.095238,0.240000,0.095238,0.335238
13,abbotje01,Jeff,Abbott,1997-06-10,2001-09-29,233.0,596.0,82.0,157.0,33.0,...,0.793991,0.051502,1,0,1997.0,2001.0,0.263423,0.305085,0.416107,0.721192
15,abbotku01,Kurt,Abbott,1993-09-07,2001-04-13,702.0,2044.0,273.0,523.0,109.0,...,0.061254,0.008547,1,1,1993.0,2001.0,0.255871,0.302200,0.422701,0.724901
21,abercre01,Reggie,Abercrombie,2006-04-04,2008-09-28,180.0,386.0,65.0,86.0,20.0,...,0.744444,0.000000,1,1,2006.0,2008.0,0.222798,0.270784,0.354922,0.625706


In [23]:
# Make a function that will create a new column to honor Jackie Robinson, the first African American Major league Baseball Player
def first_aap(col):
    if col == 'robinja02':
        return 1
    else:
        return 0
    
# Apply `first_aap` to `df['playerID']`    
df['first_aap'] = df['playerID'].apply(first_aap)

# Retrieve `df` columns
print(df.columns)

Index(['playerID', 'nameFirst', 'nameLast', 'debut', 'finalGame', 'G', 'AB',
       'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'BB', 'SO', 'HBP', 'SH', 'SF',
       'Years_Played', 'Af', 'Ef', 'DPf', 'Most Valuable Player', 'AS_games',
       'G_all', 'G_p', 'G_c', 'G_1b', 'G_2b', 'G_3b', 'G_ss', 'G_lf', 'G_cf',
       'G_rf', 'G_of', 'G_dh', 'G_p_percent', 'G_c_percent', 'G_1b_percent',
       'G_2b_percent', 'G_3b_percent', 'G_ss_percent', 'G_lf_percent',
       'G_cf_percent', 'G_rf_percent', 'G_of_percent', 'G_dh_percent',
       'bats_R', 'throws_R', 'debutYear', 'finalYear', 'AVE', 'OBP',
       'Slug_Percent', 'OPS', 'first_aap'],
      dtype='object')


In [24]:
# Create training data
# Players who already won the 'Most Valuable Player' award
df_hitters = df[df['Most Valuable Player'] > 0]
df_hitters['Most Valuable Player'].count()

68

In [25]:
# create test data
# Players who are eligible for 'Most Valuable Player' award
# df_eligible = df[df['Most Valuable Player'] <= 0]
df_eligible = df
df_eligible['Most Valuable Player'].count()

8175

In [26]:
# Select columns to use for models, and identification columns
num_cols_hitters = ['playerID', 'nameFirst', 'nameLast', 'debut', 'finalGame', 'G', 'AB',
       'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'BB', 'SO', 'HBP', 'SH', 'SF',
       'Years_Played', 'Af', 'Ef', 'DPf', 'Most Valuable Player', 'AS_games',
       'G_all', 'G_p', 'G_c', 'G_1b', 'G_2b', 'G_3b', 'G_ss', 'G_lf', 'G_cf',
       'G_rf', 'G_of', 'G_dh', 'G_p_percent', 'G_c_percent', 'G_1b_percent',
       'G_2b_percent', 'G_3b_percent', 'G_ss_percent', 'G_lf_percent',
       'G_cf_percent', 'G_rf_percent', 'G_of_percent', 'G_dh_percent',
       'bats_R', 'throws_R', 'debutYear', 'finalYear', 'AVE', 'OBP',
       'Slug_Percent', 'OPS', 'first_aap']

# Create a new DataFrame (`data`) from the `df_hitters` using the columns above
data = df_hitters[num_cols_hitters]

data.head()

,playerID,nameFirst,nameLast,debut,finalGame,G,AB,R,H,2B,...,G_dh_percent,bats_R,throws_R,debutYear,finalYear,AVE,OBP,Slug_Percent,OPS,first_aap
1,aaronha01,Hank,Aaron,1954-04-13,1976-10-03,3298.0,12364.0,2174.0,3771.0,624.0,...,0.060946,1,1,1954.0,1976.0,0.304998,0.373386,0.554513,0.927899,0
196,allendi01,Dick,Allen,1963-09-03,1977-06-19,1749.0,6332.0,1099.0,1848.0,320.0,...,0.001715,1,1,1963.0,1977.0,0.291851,0.377085,0.533639,0.910724,0
596,bagweje01,Jeff,Bagwell,1991-04-08,2005-10-02,2150.0,7797.0,1517.0,2314.0,488.0,...,0.004651,1,1,1991.0,2005.0,0.296781,0.407486,0.540336,0.947822,0
962,baylodo01,Don,Baylor,1970-09-18,1988-10-01,2292.0,8198.0,1236.0,2135.0,366.0,...,0.561518,1,1,1970.0,1988.0,0.260429,0.341134,0.435594,0.776728,0
1095,bellge02,George,Bell,1981-04-09,1993-10-02,1587.0,6123.0,814.0,1702.0,308.0,...,0.214241,1,1,1981.0,1993.0,0.277968,0.316125,0.469214,0.785340,0


In [27]:
import plotly as py
import plotly.graph_objs as go
from plotly import tools

py.offline.init_notebook_mode(connected=True)

trace1 = go.Histogram(x=data['H'], name ='Distribution of Hits')
trace2 = go.Histogram(x=data['HR'], name='Distribution of Home Runs')
trace3 = go.Histogram(x=data['AVE'], name='Distribution of Batting Average')
trace4 = go.Histogram(x=data['G'], name='Distribution of Total Games')

fig = tools.make_subplots(rows=2, cols=2)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 2, 2)

fig['layout'].update(height=800, width=800, title='Distribution of various stats')

py.offline.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



In [28]:
# Create `target` Series
target = data['Most Valuable Player']

# Create `features` DataFrame
features = data.drop(['playerID', 'nameFirst', 'nameLast', 'debut', 'debutYear', 'finalGame', 'finalGame', 'finalYear', 'Most Valuable Player'], axis=1)

In [29]:
target.count()

68

In [30]:
# Import RandomForestClassifier from sklearn
from sklearn.cross_validation import cross_val_predict, KFold
from sklearn.ensemble import RandomForestClassifier

# Create penalty dictionary
penalty = {
    0: 100,
    1: 1
}

# Create Random Forest model
rf = RandomForestClassifier(random_state=1,n_estimators=10, max_depth=1, min_samples_leaf=1, class_weight='balanced')

# Create an instance of the KFold class
kf = KFold(features.shape[0], random_state=1)

# Create predictions using cross validation
predictions_rf = cross_val_predict(rf, features, target, cv=kf)

# Convert predictions to NumPy array
np_predictions_rf = np.asarray(predictions_rf)

/Users/sureshdata/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.



In [31]:
# Create a new DataFrame from `df_eligible` using `num_col_hitters`
new_data = df_eligible[num_cols_hitters]

new_data = new_data.fillna(0)

# Create a new features DataFrame
new_features = new_data.drop(['playerID', 'nameFirst', 'nameLast', 'debut', 'debutYear', 'finalGame', 'finalGame', 'finalYear', 'Most Valuable Player'], axis=1)

In [32]:
# Fit the Random Forest model
rf.fit(features, target)

# Estimate probabilities of Hall of Fame induction
probabilities = rf.predict_proba(new_features)

In [33]:
# Create dateframe for top most active valuable players
active_mvp_df = pd.DataFrame(columns=df_eligible.columns.values.tolist())

# Convert predictions to a DataFrame
mvp_predictions = pd.DataFrame(probabilities[:,1])

# Sort the DataFrame (descending)
mvp_predictions = mvp_predictions.sort_values(0, ascending=False)

mvp_predictions['Probability'] = mvp_predictions[0]

# Print 10 highest probability 'Most Valuable Players'

import datetime
now = datetime.datetime.now()
counter = 0

# Sort the predictions based on the probability score
# The higher probability score a player gets, the more chances to win 'Most Valuable Player' award
for i, row in mvp_predictions.sort_values(by='Probability', ascending=False).iterrows():

    # probability score of player to win 'Most Valuable Player' Award
    prob = str(row['Probability'])
 
    if now.year - int(df_eligible.iloc[i]['finalYear']) < 10:
        counter += 1
        active_mvp_df = active_mvp_df.append(df_eligible.iloc[i])

    if counter >= 10:
        # find the top 10 most active valuable players and then break
        break

# drop if there are any columns that may not be necessary for further analysis
active_mvp_df = active_mvp_df.drop(columns=['debut', 'finalGame', 'debutYear', 'finalYear'])
  

In [34]:
# top 10 active players with more chances of winning 'Most Valuable Player' Award
active_mvp_df

,playerID,nameFirst,nameLast,G,AB,R,H,2B,3B,HR,...,G_rf_percent,G_of_percent,G_dh_percent,bats_R,throws_R,AVE,OBP,Slug_Percent,OPS,first_aap
6694,griffke02,Ken,Griffey,2671.0,9801.0,1662.0,2781.0,524.0,38.0,630.0,...,0.086484,0.891801,0.074878,0,0,0.283747,0.369250,0.537802,0.907052,0
17447,troutmi01,Mike,Trout,811.0,2997.0,600.0,917.0,175.0,37.0,168.0,...,0.020962,0.955610,0.040691,1,1,0.305973,0.405284,0.557224,0.962508,0
11550,mesame01,Melky,Mesa,8.0,15.0,2.0,6.0,2.0,0.0,0.0,...,0.125000,0.625000,0.125000,1,1,0.400000,0.437500,0.533333,0.970833,0
16430,stantmi03,Giancarlo,Stanton,827.0,2980.0,453.0,792.0,170.0,10.0,208.0,...,0.958888,0.958888,0.010883,1,1,0.265772,0.356872,0.538926,0.895798,0
6791,guerrvl01,Vladimir,Guerrero,2147.0,8155.0,1328.0,2590.0,477.0,46.0,449.0,...,0.747555,0.748952,0.236609,1,1,0.317597,0.378629,0.552544,0.931173,0
4874,edmonji01,Jim,Edmonds,2011.0,6858.0,1251.0,1949.0,437.0,25.0,393.0,...,0.023869,0.928394,0.010443,0,0,0.284194,0.375439,0.527122,0.902560,0
17504,turnetr01,Trea,Turner,100.0,347.0,58.0,114.0,15.0,8.0,14.0,...,0.000000,0.450000,0.000000,1,1,0.328530,0.361413,0.538905,0.900318,0
1818,braunry02,Ryan,Braun,1354.0,5249.0,879.0,1597.0,317.0,43.0,285.0,...,0.196455,0.892171,0.009601,1,1,0.304248,0.366512,0.543913,0.910425,0
1214,berkmla01,Lance,Berkman,1879.0,6491.0,1146.0,1905.0,422.0,30.0,366.0,...,0.198510,0.530601,0.055349,0,0,0.293483,0.405990,0.536897,0.942887,0
14069,ramirma02,Manny,Ramirez,2302.0,8244.0,1544.0,2574.0,547.0,20.0,555.0,...,0.392702,0.841877,0.144222,1,1,0.312227,0.410477,0.585395,0.995872,0


In [35]:
# Create a trace
trace = go.Scatter(
   x = data['HR'],
   y = data['AVE'],
   name = 'Most Valuable Player',
   mode = 'markers',
   marker = dict(
       size = 10,
       color = 'rgb(255, 0, 0)',
       line = dict(
           width = 2,
       )
   ))

trace1 = go.Scatter(
   x = active_mvp_df['HR'],
   y = active_mvp_df['AVE'],
   mode = 'markers',
   name = 'Potential Most Valuable Player',    
   marker = dict(
       size = 10,
       color = 'rgb(0, 0, 255)',
       line = dict(
           width = 2,
       )
   ))

layout = dict(title = 'Career Home Runs vs. Career Batting Average',
             yaxis = dict(zeroline = False,title= 'Career Batting Average'),
             xaxis = dict(zeroline = False,title= 'Career Home Runs')
            )

fig = dict(data=[trace, trace1], layout=layout)

# Plot and embed in ipython notebook!
py.offline.iplot(fig)

In [36]:
# Create a trace
trace = go.Scatter(
   x = data['H'],
   y = data['AVE'],
   name = 'Most Valuable Player',
   mode = 'markers',
   marker = dict(
       size = 10,
       color = 'rgb(255, 0, 0)',
       line = dict(
           width = 2,
       )
   ))

trace1 = go.Scatter(
   x = active_mvp_df['H'],
   y = active_mvp_df['AVE'],
   mode = 'markers',
   name = 'Potential Most Valuable Player',    
   marker = dict(
       size = 10,
       color = 'rgb(0, 0, 255)',
       line = dict(
           width = 2,
       )
   ))

layout = dict(title = 'Career Hits vs. Career Batting Average',
             yaxis = dict(zeroline = False,title= 'Career Batting Average'),
             xaxis = dict(zeroline = False,title= 'Career Hits')
            )

fig = dict(data=[trace, trace1], layout=layout)

# Plot and embed in ipython notebook!
py.offline.iplot(fig)

In [37]:
# Conclusion : The higher probably score a player gets, 
#              the better chances of winning 'Most Valuable Player' Award